In [19]:
import numpy as np
from collections import deque

class Node:
    def __init__(self, state, parent, action):
        self.state = state
        self.parent = parent
        self.action = action

class Puzzle:
    def __init__(self, start, goal):
        self.start = (tuple(map(tuple, start)), self.find_empty(start))  # Dynamically find empty space
        self.goal = (tuple(map(tuple, goal)), self.find_empty(goal))
        self.solution = None
        self.num_explored = 0
        self.total_states_generated = 0  # Counter for total states generated

    def find_empty(self, state):
        # Find the position of the empty space (0)
        for i in range(3):
            for j in range(3):
                if state[i][j] == 0:
                    return (i, j)

    def neighbors(self, state):
        mat, (row, col) = state
        results = []
        directions = [(1, 0, 'down'), (-1, 0, 'up'), (0, 1, 'right'), (0, -1, 'left')]

        for dr, dc, action in directions:
            new_row, new_col = row + dr, col + dc
            if 0 <= new_row < 3 and 0 <= new_col < 3:
                mat1 = np.copy(mat)
                # Swap the empty space (0) with the adjacent number
                mat1[row][col], mat1[new_row][new_col] = mat1[new_row][new_col], mat1[row][col]
                results.append((action, (tuple(map(tuple, mat1)), (new_row, new_col))))
        return results

    def solve(self):
        start_node = Node(state=self.start, parent=None, action=None)
        queue = deque([start_node])  # Use deque for efficient pops from the left
        explored = set()  # Use a set for explored states

        while queue:
            node = queue.popleft()  # Dequeue the first node
            self.num_explored += 1

            # Check if the current node's state is the goal
            if node.state[0] == self.goal[0]:
                actions = []
                cells = []
                while node.parent is not None:
                    actions.append(node.action)
                    cells.append(node.state)
                    node = node.parent
                actions.reverse()
                cells.reverse()
                self.solution = (actions, cells)
                return  # Found a solution

            # Mark the state as explored
            explored.add(node.state[0])

            # Explore neighbors
            for action, state in self.neighbors(node.state):
                if state[0] not in explored and all(node.state[0] != state[0] for node in queue):
                    child = Node(state=state, parent=node, action=action)
                    queue.append(child)  # Enqueue the child node
                    self.total_states_generated += 1  # Increment the total states generated counter

    def print_solution(self):
        if self.solution is None:
            print("No solution found.")
            return

        print("Start State:\n", np.array(self.start[0]), "\n")
        print("Goal State:\n", np.array(self.goal[0]), "\n")
        print("\nStates Explored: ", self.num_explored)
        print("Total States Generated: ", self.total_states_generated, "\n")

        print("Actions Taken to Reach the Goal:\n")
        for action, cell in zip(self.solution[0], self.solution[1]):
            print("Action: ", action)
            print(np.array(cell[0]), "\n")
        print("Goal Reached!!")

# Example usage
start = np.array([[1, 2, 3], [0, 4, 6], [7, 5, 8]])
goal = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 0]])

p = Puzzle(start, goal)
p.solve()
p.print_solution()

Start State:
 [[1 2 3]
 [0 4 6]
 [7 5 8]] 

Goal State:
 [[1 2 3]
 [4 5 6]
 [7 8 0]] 


States Explored:  14
Total States Generated:  26 

Actions Taken to Reach the Goal:

Action:  right
[[1 2 3]
 [4 0 6]
 [7 5 8]] 

Action:  down
[[1 2 3]
 [4 5 6]
 [7 0 8]] 

Action:  right
[[1 2 3]
 [4 5 6]
 [7 8 0]] 

Goal Reached!!


In [ ]:
from collections import deque

GOAL_STATE = (1, 2, 3, 4, 5, 6, 7, 8, 0)

def find_empty(state):
    return state.index(0)

def get_neighbors(state):
    neighbors = []
    empty_index = find_empty(state)
    row, col = divmod(empty_index, 3)
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
    for dr, dc in directions:
        new_row, new_col = row + dr, col + dc
        if 0 <= new_row < 3 and 0 <= new_col < 3:
            new_index = new_row * 3 + new_col
            new_state = list(state)
            new_state[empty_index], new_state[new_index] = new_state[new_index], new_state[empty_index]
            neighbors.append(tuple(new_state))
    return neighbors

def dfs(initial_state):
    stack = [(initial_state, [])]
    visited = set()
    visited.add(initial_state)
    visited_count = 1  # Initialize visited count
    while stack:
        current_state, path = stack.pop()
        if current_state == GOAL_STATE:
            return path, visited_count  # Return path and count
        for neighbor in get_neighbors(current_state):
            if neighbor not in visited:
                visited.add(neighbor)
                stack.append((neighbor, path + [neighbor]))
                visited_count += 1  # Increment visited count
    return None, visited_count  # Return count if no solution found

def input_start_state():
    print("Enter the starting state as 9 numbers (0 for the empty space):")
    input_state = input("Format: 1 2 3 4 5 6 7 8 0\n")
    numbers = list(map(int, input_state.split()))
    if len(numbers) != 9 or set(numbers) != set(range(9)):
        print("Invalid input. Please enter numbers from 0 to 8 with no duplicates.")
        return input_start_state()
    return tuple(numbers)

def print_matrix(state):
    for i in range(0, 9, 3):
        print(state[i:i+3])

if __name__ == "__main__":
    initial_state = input_start_state()
    print("Initial state:")
    print_matrix(initial_state)
    solution, visited_count = dfs(initial_state)
    print(f"Number of states visited: {visited_count}")
    if solution:
        print("\nSolution found with the following steps:")
        for step in solution:
            print_matrix(step)
            print()
    else:
        print("No solution found.")



Enter the starting state as 9 numbers (0 for the empty space):
Format: 1 2 3 4 5 6 7 8 0
1 2 3 0 4 6 7 5 8
Initial state:
(1, 2, 3)
(0, 4, 6)
(7, 5, 8)
